In [21]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [22]:
#ITPトラッキング
gsheet_url = "https://docs.google.com/spreadsheets/d/128vzSG0oHymvoaRsRpQ-5F3l0yzFJHOivWEv1MLt8zM/edit#gid=0"

In [23]:
#日付パラメータ(遡る日数を指定するためのパラメーター)
start = 2
end = 16

# ITPトラッキング

1　imp/ctv/R/など

In [24]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select 
        concat_ws('-', year, month, day)ymd
        ,user_agent_os_major
        ,user_agent_browser_family
        ,user_agent_os_family
        ,sum(case when action_type = 'I' then 1 else 0 end) imp
        ,sum(case when action_type = 'C' then 1 else 0 end) click
        ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
        ,sum(winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when action_type = 'MTV' then 1 else 0 end) MTV
        ,sum(case when action_type = 'NV' then 1 else 0 end) NV
        ,sum(case when action_type = 'R' then 1 else 0 end) R
        from dsplog.adserver ads
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
        and ads.advertiser_id not in (8521)
        group by 1,2,3,4
        having imp > 0
        order by 1,2
        """
        )
        
        df_ios = ir.sql_to_pandas(query)
        df_ios = df_ios.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 0
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_ios, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

2　res

In [25]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day)ymd
            ,user_agent_os_major
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(*)*100 res
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
            and bidden=true
            and advertiser_id not in(3879,6295,6253,5889) 
            group by 1,2,3,4 
            order by 1,2
            """
        )
        df_res = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 1
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_res, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

3　req_uu ※sync済み

In [26]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day)ymd
            ,user_agent_os_major
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(distinct smn_uid)*100 uu
            from dsplog_sampling.bid_response_uid_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
            group by 1,2,3,4 
            order by 1,2
            """
        )
        df_req_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_req_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

4 res uu

In [27]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            , user_agent_os_major
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(distinct smn_uid)*100 res_uu
            from dsplog_sampling.bid_response_uid_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
            and bidden=true
            and advertiser_id not in(3879,6295,6253,5889) 
            group by 1,2,3,4 
            order by 1,2
            """
        )
        df_res_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 3
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_res_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

5 R uu

In [28]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select 
            concat_ws('-', year, month, day) ymd
            , user_agent_os_major
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(distinct smn_uid) uu_r
            from dsplog.adserver
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
            and action_type in('R')
            group by 1,2,3,4 
            order by 1,2
            """
        )
        df_r_uu = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 4
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_r_uu, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

6 req　※sync済み

In [29]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            , user_agent_os_major
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(smn_uid)*100 req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
            group by 1,2,3,4 
            order by 1,2
            """
        )
        df_req = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 5
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

7　⇒　なし

8 req ※SSP別

In [30]:
for i in range(start,end):

    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,case when ssp_id=1 then 'Google' else 'etc' end ssp
            ,user_agent_os_major
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(smn_uid)*100 as req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
            group by 1,2,3,4,5 
            order by 1,2,3
            """
        )
        df_req_ssp = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 6
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_req_ssp, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

9　⇒　なし

10　ALL　req

In [31]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            , user_agent_os_major
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(1)*100 req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
            group by 1,2,3,4 
            order by 1,2
            """
        )
        df_all_req = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 7
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [32]:
#1　imp/ctv/R/など ただし、552	株式会社パピレス　4396	ヤフー株式会社（ヤフオク）　5928	ヤフー株式会社（Yahoo!ショッピング）除く

In [33]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select 
        concat_ws('-', year, month, day)ymd
        ,user_agent_os_major
        ,user_agent_browser_family
        ,user_agent_os_family
        ,sum(case when action_type = 'I' then 1 else 0 end) imp
        ,sum(case when action_type = 'C' then 1 else 0 end) click
        ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
        ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when action_type = 'MTV' then 1 else 0 end) MTV
        ,sum(case when action_type = 'NV' then 1 else 0 end) NV
        ,sum(case when action_type = 'R' then 1 else 0 end) R
        from dsplog.adserver
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')) 
        and advertiser_id not in(552,4396,5928,8521) 
        group by 1,2,3,4
        having imp > 0
        order by 1,2
        """
        )
        
        df_imp = ir.sql_to_pandas(query)
        df_imp = df_imp.astype({'win_price':float})
        gs = GSheet()
        sheet_index = 8
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_imp, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

# ITPトラッキング(マイナー)

In [34]:
gsheet_url2 = "https://docs.google.com/spreadsheets/d/1AunOQPSLGrsLyEG8WdIFX5jzs0Tgzm2ufpChEzk2Fag/edit#gid=1454753137"#マイナー

10 req

In [35]:
for i in range(start,end):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select 
            concat_ws('-', year, month, day) ymd
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_major else '0' end user_agent_os_major
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_minor else '0' end user_agent_os_minor
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(1)*100 
            from dsplog_sampling.bid_response_time_one_percent_sampling 
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))
            and user_agent_os_family in('iOS') 
            and user_agent_browser_family in('Mobile Safari') 
            group by 1,2,3,4,5 
            order by 1,2,3
        """
        )
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            df_req = ir.sql_to_pandas(query)
            gs = GSheet()
            sheet_index = 6
            col_offset=0
            gs.append_data(gsheet_url2, sheet_index, df_req, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:ailab_tools.smn.impala_clien

In [ ]:
#以下更新、取得してない

1　imp/ctv/R/など

%%slack_notify @yusuke_akada
for i in range(2,16):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select 
            concat_ws('-', year, month, day) as ymd
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_major else '0' end user_agent_os_major
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_minor else '0' end user_agent_os_minor
            ,user_agent_browser_family
            ,user_agent_os_family
            ,sum(case when action_type = 'I' then 1 else 0 end) imp
            ,sum(case when action_type = 'C' then 1 else 0 end) click
            ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
            ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
            ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
            ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
            ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
            ,sum(case when action_type = 'MTV' then 1 else 0 end) MTV
            ,sum(case when action_type = 'NV' then 1 else 0 end) NV
            ,sum(case when action_type = 'R' then 1 else 0 end) R
            from 
            dsplog.adserver
            where concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari'))
            group by 1,2,3,4,5 
            order by 1,2,3
            """
        )
        
        df_imp = ir.sql_to_pandas(query)
        df_imp = df_imp.astype({'win_price':float})
        gs = GSheet()
        sheet_index = 0
        col_offset=0
        gs.append_data(gsheet_url2, sheet_index, df_imp, col_offset)

2 imp/ctv/R/など(2)

%%slack_notify @yusuke_akada
for i in range(2,16):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select
            concat_ws('-', year, month, day)ymd
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_major else '0' end user_agent_os_major
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_minor else '0' end user_agent_os_minor
            ,user_agent_browser_family,user_agent_os_family,case when ad_type in(1,2,3) then 'non_rtg' when ad_type in(0,4) then 'rtg' else 'unknown' end ad_type
            ,sum(case when action_type = 'I' then 1 else 0 end) imp
            ,sum(case when action_type = 'C' then 1 else 0 end) click
            ,sum(case when action_type = 'I' then winning_price else 0 end)/1000000 win_price
            ,sum(case when action_type = 'CTV1' then 1 else 0 end) ctv1
            ,sum(case when action_type = 'CTV2' then 1 else 0 end) ctv2
            ,sum(case when action_type = 'VTV1' then 1 else 0 end) vtv1
            ,sum(case when action_type = 'VTV2' then 1 else 0 end) vtv2
            from 
            dsplog.adserver  a
            inner join dm.hierarchies b using(target_id) 
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari') 
            group by 1,2,3,4,5,6
            order by 1,2,3
            """
        )
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            df_imp2 = ir.sql_to_pandas(query)
            df_imp2 = df_imp.astype({'win_price':float})
            gs = GSheet()
            sheet_index = 1
            col_offset=0
            gs.append_data(gsheet_url2, sheet_index, df_imp2, col_offset)

3 req_sync

%%slack_notify @yusuke_akada
for i in range(2,16):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select
        concat_ws('-', year, month, day) ymd
        ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_major else '0' end user_agent_os_major
        ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_minor else '0' end user_agent_os_minor
        ,user_agent_browser_family
        ,user_agent_os_family,
        count(smn_uid)*100 req_sync
        from 
        dsplog_sampling.bid_response_time_one_percent_sampling
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))
        and user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')
        group by 1,2,3,4,5
        order by 1,2,3
        """
        )
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            df_req_s = ir.sql_to_pandas(query)
            gs = GSheet()
            sheet_index = 2
            col_offset=0
            gs.append_data(gsheet_url2, sheet_index, df_req_s, col_offset)

4 req_uu

%%slack_notify @yusuke_akada
for i in range(2,16):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select 
            concat_ws('-', year, month, day) ymd
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_major else '0' end user_agent_os_major
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_minor else '0' end user_agent_os_minor
            ,user_agent_browser_family
            ,user_agent_os_family
            ,count(distinct smn_uid) uu
            from 
            dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))
            and user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari')
            group by 1,2,3,4,5
            order by 1,2,3

            """
        )
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            df_req_uu = ir.sql_to_pandas(query)
            gs = GSheet()
            sheet_index = 3
            col_offset=0
            gs.append_data(gsheet_url2, sheet_index, df_req_uu, col_offset)

5 bidden

%%slack_notify @yusuke_akada
for i in range(2,16):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select
            concat_ws('-', year, month, day) ymd
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_major else '0' end user_agent_os_major
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_minor else '0' end user_agent_os_minor
            ,user_agent_browser_family
            ,user_agent_os_family
            ,case when ad_type in(1,2,3) then 'non_rtg' when ad_type in(0,4) then 'rtg' else 'unknown' end ad_type
            ,count(*) *100 bidden  
            from 
            dsplog_sampling.bid_response_time_one_percent_sampling a
            inner join 
            dm.hierarchies b using(target_id)
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and user_agent_os_family in('iOS') 
            and user_agent_browser_family in('Mobile Safari') 
            and bidden=true
            group by 1,2,3,4,5,6
            order by 1,2,3
            """
        )
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                            request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            df_bidden = ir.sql_to_pandas(query)
            gs = GSheet()
            sheet_index = 4
            col_offset=0
            gs.append_data(gsheet_url2, sheet_index, df_bidden, col_offset)

6 bidden_uu

%%slack_notify @yusuke_akada
for i in range(2,16):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select
            concat_ws('-', year, month, day) ymd
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_major else '0' end user_agent_os_major
            ,case when cast(user_agent_os_major as integer) >10 then user_agent_os_minor else '0' end user_agent_os_minor
            ,user_agent_browser_family
            ,user_agent_os_family
            ,case when ad_type in(1,2,3) then 'non_rtg' when ad_type in(0,4) then 'rtg' else 'unknown' end ad_type
            ,count(distinct smn_uid)*100 bidden  
            from 
            dsplog_sampling.bid_response_time_one_percent_sampling a
            inner join 
            dm.hierarchies b using(target_id)
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
            and user_agent_os_family in('iOS') 
            and user_agent_browser_family in('Mobile Safari') 
            and bidden=true
            group by 1,2,3,4,5,6
            order by 1,2,3
            """
        )
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
                df_bidden_uu = ir.sql_to_pandas(query)
                gs = GSheet()
                sheet_index = 5
                col_offset=0
                gs.append_data(gsheet_url2, sheet_index, df_bidden_uu, col_offset)

8,9 →無し

追加調査　2019/6/20あたりを境にGoogleのリクエストが減っている<br>
仮説①1st Price対応の影響か？　⇒なさそう

8 req ※SSP別

%%slack_notify @yusuke_akada
for i in range(3,42):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select concat_ws('-', year, month, day) ymd
            ,open_auction_type
            ,bidding_auction_type
            ,fd
            ,count(smn_uid)*100 as req
            from dsplog_sampling.bid_response_time_one_percent_sampling
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and  (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari'))
            and ssp_id = 1
            group by 1,2,3,4
            order by 1,2,3,4
            """
        )
        df_req_tmp = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 9
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_req_tmp, col_offset)

仮説②Googleだけ何か調整することがよくある。他のOSなどに変化はないか？

8 req ※SSP別

%%slack_notify @yusuke_akada
for i in range(3,31):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            select 
            concat_ws('-', year, month, day) ymd
            ,case 
            when cast(is_app as int) = 1 and user_agent_os_family in('Android') then 'Androidアプリ'
            when cast(is_app as int) = 0 and user_agent_os_family in('Android') then 'Androidウェブ'
            when cast(is_app as int) = 1 and user_agent_os_family in('iOS') then 'iOSアプリ'
            when cast(is_app as int) = 0 and user_agent_os_family in('iOS') then 'iOSウェブ'
            else 'PC'
            end as OS
            ,sum(req) req
            ,sum(smn_uid_count) sync
            from dm.domain_req 
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i}))  
            and ssp_id > 0 
            group by 1,2
            """
        )
        df_req_tmp = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 1
        col_offset=0
        gs.append_data("https://docs.google.com/spreadsheets/d/1BkN7nGKW5twslsIYNIvz8s610iYgGp3TLASvvGgRHsA/edit#gid=1968402840", sheet_index, df_req_tmp, col_offset)

追加調査2　CTVが6/30を境にios12で減っている。なぜ？？

%%slack_notify @yusuke_akada
for i in range(3,20):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select 
        concat_ws('-', year, month, day) as ymd
        ,ads.advertiser_id
        ,hie.advertiser_name
        ,ads.strategy_id
        ,hie.strategy_name
        ,user_agent_os_major
        ,count(*) as cnt
        from dsplog.adserver ads
        inner join dm.hierarchies hie on ads.target_id = hie.target_id 
        where 
        concat_ws('-',year,month,day) between to_date(days_add(now(),-{i})) and to_date(days_add(now(),-{i})) 
        and  (user_agent_os_family in('iOS') and user_agent_browser_family in('Mobile Safari'))
        and ads.user_agent_os_major in ('10','11','12')
        and ads.action_type in ('CTV1','CTV2')
        group by 1,2,3,4,5,6
        order by 1,2,3,4,5,6
        """
        )
        
        df_imp = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data("https://docs.google.com/spreadsheets/d/1BkN7nGKW5twslsIYNIvz8s610iYgGp3TLASvvGgRHsA/edit#gid=1968402840", sheet_index, df_imp, col_offset)